In [1]:
import pandas as pd
import numpy as np
import sys, os
from IPython.display import display

## Request Solar Data from NSRDB

In [32]:
# Declare all variables as strings. Spaces must be replaced with '+', i.e., change 'John Smith' to 'John+Smith'.
  # Define the lat, long of the location and the year
lat, lon, year = 34.05, -118.24, 2010
  # You must request an NSRDB api key from the link above
api_key = 'CvkzSIZGV3OxJgzOAak39Yh8AzwYnlSyJ0lKUzaa'
  # Set the attributes to extract (e.g., dhi, ghi, etc.), separated by commas.
attributes = 'ghi,dhi,dni,wind_speed,wind_direction,air_temperature,solar_zenith_angle'
  # Choose year of data
year = '2010'
  # Set leap year to true or false. True will return leap day data if present, false will not.
leap_year = 'false'
  # Set time interval in minutes, i.e., '30' is half hour intervals. Valid intervals are 30 & 60.
interval = '60'
  # Specify Coordinated Universal Time (UTC), 'true' will use UTC, 'false' will use the local time zone of the data.
  # NOTE: In order to use the NSRDB data in SAM, you must specify UTC as 'false'. SAM requires the data to be in the
  # local time zone.
utc = 'false'
  # Your full name, use '+' instead of spaces.
your_name = 'Tianxiao+Shen'
  # Your reason for using the NSRDB.
reason_for_use = 'Course+Project'
  # Your affiliation
your_affiliation = 'Columbia+University'
  # Your email address
your_email = 'ts3326@columbia.edu'
  # Please join our mailing list so we can keep you up-to-date on new developments.
mailing_list = 'false'

  # Declare url string
url = 'https://developer.nrel.gov/api/solar/nsrdb_psm3_download.csv?wkt=POINT({lon}%20{lat})&names={year}&leap_day={leap}&interval={interval}&utc={utc}&full_name={name}&email={email}&affiliation={affiliation}&mailing_list={mailing_list}&reason={reason}&api_key={api}&attributes={attr}'.format(year=year, lat=lat, lon=lon, leap=leap_year, interval=interval, utc=utc, name=your_name, email=your_email, mailing_list=mailing_list, affiliation=your_affiliation, reason=reason_for_use, api=api_key, attr=attributes)
  # Return just the first 2 lines to get metadata:
info = pd.read_csv(url, nrows=1)
  # See metadata for specified properties, e.g., timezone and elevation
timezone, elevation = info['Local Time Zone'], info['Elevation']

In [33]:
# Return all but first 2 lines of csv to get data:
df = pd.read_csv(url, skiprows=2)

  # Set the time index in the pandas dataframe:
df = df.set_index(pd.date_range('1/1/{yr}'.format(yr=year), freq=interval+'Min', periods=525600/int(interval)))

  # take a look
print('shape:',df.shape)
df.head()

shape: (8760, 12)


,Year,Month,Day,Hour,Minute,GHI,DHI,DNI,Wind Speed,Wind Direction,Temperature,Solar Zenith Angle
2010-01-01 00:00:00,2010,1,1,0,30,0,0,0,2.4,64.8,9,166.71
2010-01-01 01:00:00,2010,1,1,1,30,0,0,0,2.6,62.7,9,156.73
2010-01-01 02:00:00,2010,1,1,2,30,0,0,0,2.8,60.9,8,144.73
2010-01-01 03:00:00,2010,1,1,3,30,0,0,0,3.0,57.4,8,132.33
2010-01-01 04:00:00,2010,1,1,4,30,0,0,0,3.5,52.4,9,119.97


## Request AQI Pollutant Data from USEPA AQS

In [2]:
from urllib.request import urlopen
import json

In [8]:
def request_AQI_data(year, site, param):
    email = "ts3326@columbia.edu"
    key   = "rubymallard71"
    bdate = "%i0101"%year
    edate = "%i1231"%year
    state = "06"
    county= "037"
    
    url = 'https://aqs.epa.gov/data/api/sampleData/bySite?email={email}&key={key}&param={param}&bdate={bdate}&edate={edate}&state={state}&county={county}&site={site}'.format(email=email, key=key, param=param, bdate=bdate, edate=edate, state=state, county=county, site=site)

    u = urlopen(url)
    resp = json.loads(u.read().decode('utf-8'))
    
    data = pd.DataFrame(columns = resp["Data"][0].keys())
    for i in resp["Data"]:
        temp = pd.DataFrame(i, index=[0])
        data = data.append(temp, ignore_index=True)
    data_sorted = data.drop(['state_code', 'county_code', 'site_number', 'parameter_code', 'poc', 'latitude', 'longitude', 'datum', 'parameter', 'date_gmt', 'time_gmt', 'units_of_measure_code', 'sample_duration_code', 'uncertainty', 'qualifier', 'method_type', 'method', 'method_code', 'state', 'county', 'date_of_last_change', 'cbsa_code'],axis=1)
    #data_sorted = data_sorted.drop(data_sorted[data_sorted["sample_duration"] == "24 HOUR"].index)
    data_sorted = data_sorted.sort_values(["date_local", "time_local"])
    data_sorted = data_sorted.reset_index()
    data_sorted.to_csv("%i_%i_%i.csv"%(year,param,site))
    
    return print("%i_%i_%i.csv"%(year,param,site) + " completed")
    

In [88]:
for i in [2016,2017,2018]:
    request_AQI_data(i, 5005, 42602)

2016_42602_5005.csv completed
2017_42602_5005.csv completed
2018_42602_5005.csv completed


## Request LA Historical Weather Data

In [68]:
from urllib.request import urlopen
import json

In [69]:
def request_Weather_data(year, month, days):
    q       = "Los+Angeles"
    date    = "%i-%s-01"% (year,month)
    enddate = "%i-%s-%i"% (year,month,days)
    tp      = 1
    format  = "json"
    key     = "a61d95712c3245f88cc172632212312"
    url = 'https://api.worldweatheronline.com/premium/v1/past-weather.ashx?q={q}&date={date}&enddate={enddate}&tp={tp}&format={format}&key={key}'.format(q=q, date=date, enddate=enddate, tp=tp, format=format, key=key)

    u = urlopen(url)
    resp = json.loads(u.read().decode('utf-8'))
    
    weather_data = pd.DataFrame()
    for i in resp['data']["weather"]:
        for j in i["hourly"]:
            temp = pd.DataFrame(j, index=[0])
            temp["date"] = i["date"]
            weather_data = weather_data.append(temp, ignore_index=True)
            
    data_sorted = weather_data.drop(columns = ['tempF', 'windspeedMiles', 'winddir16Point', 'weatherCode', 'weatherIconUrl', 'weatherDesc', 'precipInches','visibilityMiles','pressureInches', 'HeatIndexC', 'HeatIndexF','DewPointF', 'WindChillC', 'WindChillF', 'WindGustMiles', 'FeelsLikeC', 'FeelsLikeF', 'uvIndex'])
    
    data_sorted = data_sorted.sort_values(["date", "time"])
    data_sorted = data_sorted.reset_index()
    
    return data_sorted
    

In [83]:
months = ["01","02","03","04","05","06","07","08","09","10","11","12"]
days = [31,28,31,30,31,30,31,31,30,31,30,31]

data = pd.DataFrame()
for y in [2019]:
    for i in range(12):
        data = data.append(request_Weather_data(y,months[i],days[i]))

In [84]:
data['time']=data['time'].astype(int)
data = data.sort_values(["date", "time"])
data = data.set_index("date")

In [85]:
data = data.drop(columns = ["index"])

In [86]:
data.to_csv("%i_%s_weather.csv"%(y,months[i]))

In [13]:
q       = "Los+Angeles"
date    = "2016-01-01"
enddate = "2020-12-31"
tp      = 1
format  = "xml"
key     = "a61d95712c3245f88cc172632212312"
url = 'https://api.worldweatheronline.com/premium/v1/past-weather.ashx?q={q}&date={date}&enddate={enddate}&tp={tp}&format={format}&key={key}'.format(q=q, date=date, enddate=enddate, tp=tp, format=format, key=key)

u = urlopen(url)
resp = json.loads(u.read().decode('utf-8'))


In [16]:
resp

{'data': {'request': [{'type': 'City',
    'query': 'Los Angeles, United States of America'}],
  'weather': [{'date': '2016-01-01',
    'astronomy': [{'sunrise': '06:59 AM',
      'sunset': '04:54 PM',
      'moonrise': 'No moonrise',
      'moonset': '11:29 AM',
      'moon_phase': 'Last Quarter',
      'moon_illumination': '50'}],
    'maxtempC': '18',
    'maxtempF': '64',
    'mintempC': '6',
    'mintempF': '43',
    'avgtempC': '13',
    'avgtempF': '56',
    'totalSnow_cm': '0.0',
    'sunHour': '8.7',
    'uvIndex': '4',
    'hourly': [{'time': '0',
      'tempC': '7',
      'tempF': '45',
      'windspeedMiles': '9',
      'windspeedKmph': '14',
      'winddirDegree': '10',
      'winddir16Point': 'N',
      'weatherCode': '113',
      'weatherIconUrl': [{'value': 'http://cdn.worldweatheronline.com/images/wsymbols01_png_64/wsymbol_0008_clear_sky_night.png'}],
      'weatherDesc': [{'value': 'Clear'}],
      'precipMM': '0.0',
      'precipInches': '0.0',
      'humidity': '35'

In [30]:
weather_data = pd.DataFrame()
for i in resp['data']["weather"]:
    for j in i["hourly"]:
        temp = pd.DataFrame(j, index=[0])
        temp["date"] = i["date"]
        weather_data = weather_data.append(temp, ignore_index=True)

In [34]:
resp['data']["weather"][0]["hourly"][0].keys()

dict_keys(['time', 'tempC', 'tempF', 'windspeedMiles', 'windspeedKmph', 'winddirDegree', 'winddir16Point', 'weatherCode', 'weatherIconUrl', 'weatherDesc', 'precipMM', 'precipInches', 'humidity', 'visibility', 'visibilityMiles', 'pressure', 'pressureInches', 'cloudcover', 'HeatIndexC', 'HeatIndexF', 'DewPointC', 'DewPointF', 'WindChillC', 'WindChillF', 'WindGustMiles', 'WindGustKmph', 'FeelsLikeC', 'FeelsLikeF', 'uvIndex'])

In [45]:
weather_data = pd.DataFrame()
for i in resp['data']["weather"]:
    for j in i["hourly"]:
        temp = pd.DataFrame(j, index=[0])
        temp["date"] = i["date"]
        weather_data = weather_data.append(temp, ignore_index=True)
data_sorted = weather_data.drop(columns = ['tempF', 'windspeedMiles', 'winddir16Point', 'weatherCode', 'weatherIconUrl', 'weatherDesc', 'precipInches','visibilityMiles','pressureInches', 'HeatIndexC', 'HeatIndexF','DewPointF', 'WindChillC', 'WindChillF', 'WindGustMiles', 'FeelsLikeC', 'FeelsLikeF', 'uvIndex'])    #data_sorted = data_sorted.drop(data_sorted[data_sorted["sample_duration"] == "24 HOUR"].index)


In [46]:
data_sorted

,time,tempC,windspeedKmph,winddirDegree,precipMM,humidity,visibility,pressure,cloudcover,DewPointC,WindGustKmph,date
0,0,7,14,10,0.0,35,10,1020,0,-8,30,2016-01-01
1,100,7,14,17,0.0,34,10,1020,1,-8,30,2016-01-01
2,200,7,14,25,0.0,34,10,1020,1,-8,30,2016-01-01
3,300,6,14,32,0.0,34,10,1020,2,-8,30,2016-01-01
4,400,6,15,24,0.0,34,10,1020,2,-9,30,2016-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...
833,1700,16,10,326,0.0,32,10,1025,1,-1,20,2016-02-04
834,1800,14,13,352,0.0,35,10,1026,0,-1,27,2016-02-04
835,1900,13,12,245,0.0,33,10,1026,1,-3,25,2016-02-04
836,2000,12,11,138,0.0,32,10,1026,2,-4,23,2016-02-04


In [ ]:
data = pd.DataFrame(columns = resp["Data"][0].keys())